In [6]:
import numpy as np
from scipy.linalg import norm
from scipy.optimize import broyden1

# Define the system of nonlinear equations
def system(X):
    x, y = X
    return np.array([x**2 + y**2 - 4, np.exp(x) + y - 1])

# Jacobian function (for reference, though not used in quasi-Newton)
def jacobian(X):
    x, y = X
    return np.array([[2*x, 2*y], [np.exp(x), 1]])

# Newton-like methods

# Lazy update method
def lazy_newton(system, X0, tol=1e-6, max_iter=100, lazy_update_step=5):
    X = np.array(X0, dtype=float)
    B = jacobian(X0)  # Initial Jacobian
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1  # Solution found
        
        if i % lazy_update_step == 0:  # Update the Jacobian lazily every few iterations
            B = jacobian(X)
        
        try:
            dX = np.linalg.solve(B, -F)
        except np.linalg.LinAlgError:
            print(f"Jacobian matrix is singular at iteration {i}")
            return None, i
        
        X = X + dX
    
    return None, max_iter  # Solution not found

# Broyden's method
def broyden_method(system, X0, tol=1e-6, max_iter=100):
    B = jacobian(X0)  # Initial Jacobian approximation
    X = np.array(X0, dtype=float)
    
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1
        
        try:
            dX = np.linalg.solve(B, -F)
        except np.linalg.LinAlgError:
            print(f"Jacobian matrix is singular at iteration {i}")
            return None, i
        
        X_new = X + dX
        F_new = system(X_new)
        
        # Update Broyden's Jacobian approximation
        y = F_new - F
        B = B + np.outer((y - B @ dX), dX) / np.dot(dX, dX)
        
        X = X_new
    
    return None, max_iter  # Solution not found

# Initial guesses
initial_guesses = [(1, 1), (1, -1), (0, 0)]




In [7]:
# Updated system to handle overflow in exp
def system_safe(X):
    x, y = X
    try:
        exp_x = np.exp(x)
    except OverflowError:
        exp_x = np.inf
    return np.array([x**2 + y**2 - 4, exp_x + y - 1])

# Re-run the methods with safe system definition





In [8]:
# Updated system to cap the exponential values to avoid overflow
def system_capped(X):
    x, y = X
    exp_x = np.exp(min(x, 700))  # Cap the value of x in exp(x) to avoid overflow
    return np.array([x**2 + y**2 - 4, exp_x + y - 1])

# Re-run the methods with capped system definition

results = {}

for guess in initial_guesses:
    lazy_solution, lazy_iters = lazy_newton(system_capped, guess)
    broyden_solution, broyden_iters = broyden_method(system_capped, guess)
    
    results[guess] = {
        'Lazy': (lazy_solution, lazy_iters),
        'Broyden': (broyden_solution, broyden_iters)
    }

results

Jacobian matrix is singular at iteration 0
Jacobian matrix is singular at iteration 0


/var/folders/2y/4ydf1r5x3lz1mksx0rhvm69c0000gn/T/ipykernel_42367/241373372.py:13: RuntimeWarning: overflow encountered in exp
  return np.array([[2*x, 2*y], [np.exp(x), 1]])


{(1, 1): {'Lazy': (None, 100),
  'Broyden': (array([-1.81626406,  0.83736777]), 13)},
 (1, -1): {'Lazy': (array([ 1.00416874, -1.72963729]), 9),
  'Broyden': (array([ 1.00416874, -1.7296373 ]), 7)},
 (0, 0): {'Lazy': (None, 0), 'Broyden': (None, 0)}}

In [14]:
import numpy as np
from scipy.linalg import norm

# Define the nonlinear system
def system(X):
    x, y, z = X
    f1 = x + np.cos(x * y * z) - 1
    f2 = (1 - x)**(1/4) + y + 0.05 * z**2 - 0.15 * z - 1
    f3 = -x**2 - 0.1 * y**2 + 0.01 * y + z - 1
    return np.array([f1, f2, f3])

# Jacobian for Newton's method
def jacobian(X):
    x, y, z = X
    J = np.zeros((3, 3))
    
    # Partial derivatives for f1
    J[0, 0] = 1 - y * z * np.sin(x * y * z)  # df1/dx
    J[0, 1] = -x * z * np.sin(x * y * z)     # df1/dy
    J[0, 2] = -x * y * np.sin(x * y * z)     # df1/dz
    
    # Partial derivatives for f2
    J[1, 0] = -0.25 * (1 - x)**(-3/4)       # df2/dx
    J[1, 1] = 1                              # df2/dy
    J[1, 2] = 0.1 * z - 0.15                 # df2/dz
    
    # Partial derivatives for f3
    J[2, 0] = -2 * x                         # df3/dx
    J[2, 1] = -0.2 * y + 0.01                # df3/dy
    J[2, 2] = 1                              # df3/dz
    
    return J

# Newton's method
def newton_method(system, jacobian, X0, tol=1e-6, max_iter=100):
    X = np.array(X0, dtype=float)
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1
        
        J = jacobian(X)
        try:
            dX = np.linalg.solve(J, -F)
        except np.linalg.LinAlgError:
            print(f"Jacobian is singular at iteration {i}")
            return None, i
        
        X = X + dX
    
    return None, max_iter

# Steepest Descent method
def steepest_descent(system, X0, tol=1e-6, max_iter=100, alpha=0.01):
    X = np.array(X0, dtype=float)
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1
        
        # Use the system's value as an approximation of the gradient
        X = X - alpha * F
    
    return None, max_iter

# First Steepest Descent then Newton's method
def hybrid_method(system, jacobian, X0, tol_sd=5e-2, tol_newton=1e-6, max_iter=100):
    # First use steepest descent
    X, sd_iters = steepest_descent(system, X0, tol=tol_sd, max_iter=max_iter)
    if X is None:
        return None, sd_iters
    
    # Use the result from steepest descent as the initial guess for Newton's method
    X, newton_iters = newton_method(system, jacobian, X, tol=tol_newton, max_iter=max_iter)
    
    return X, sd_iters + newton_iters

# Initial guess
initial_guess = [0.5, 0.5, 0.5]

# Run the methods
results = {}

# Newton's method
newton_solution, newton_iters = newton_method(system, jacobian, initial_guess)

# Steepest descent method
sd_solution, sd_iters = steepest_descent(system, initial_guess)

# Hybrid method (Steepest descent followed by Newton's)
hybrid_solution, hybrid_iters = hybrid_method(system, jacobian, initial_guess)

# Store the results
results['Newton'] = (newton_solution, newton_iters)
results['Steepest Descent'] = (sd_solution, sd_iters)
results['Hybrid'] = (hybrid_solution, hybrid_iters)

# Display the results
import pandas as pd
df_results = pd.DataFrame(results, index=['Solution', 'Iterations'])
df_results

,Newton,Steepest Descent,Hybrid
Solution,"[-4.240797505370743e-17, 0.10000000000014028, ...",NaN,NaN
Iterations,5,100.0,100.0


In [12]:
# Reimport necessary modules after the reset

import numpy as np
from scipy.linalg import norm
import pandas as pd

# Define the nonlinear system
def system(X):
    x, y, z = X
    f1 = x + np.cos(x * y * z) - 1
    f2 = (1 - x)**(1/4) + y + 0.05 * z**2 - 0.15 * z - 1
    f3 = -x**2 - 0.1 * y**2 + 0.01 * y + z - 1
    return np.array([f1, f2, f3])

# Jacobian for Newton's method
def jacobian(X):
    x, y, z = X
    J = np.zeros((3, 3))
    
    # Partial derivatives for f1
    J[0, 0] = 1 - y * z * np.sin(x * y * z)  # df1/dx
    J[0, 1] = -x * z * np.sin(x * y * z)     # df1/dy
    J[0, 2] = -x * y * np.sin(x * y * z)     # df1/dz
    
    # Partial derivatives for f2
    J[1, 0] = -0.25 * (1 - x)**(-3/4)       # df2/dx
    J[1, 1] = 1                              # df2/dy
    J[1, 2] = 0.1 * z - 0.15                 # df2/dz
    
    # Partial derivatives for f3
    J[2, 0] = -2 * x                         # df3/dx
    J[2, 1] = -0.2 * y + 0.01                # df3/dy
    J[2, 2] = 1                              # df3/dz
    
    return J

# Newton's method
def newton_method(system, jacobian, X0, tol=1e-6, max_iter=100):
    X = np.array(X0, dtype=float)
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1
        
        J = jacobian(X)
        try:
            dX = np.linalg.solve(J, -F)
        except np.linalg.LinAlgError:
            print(f"Jacobian is singular at iteration {i}")
            return None, i
        
        X = X + dX
    
    return None, max_iter

# Implementing a basic line search for the steepest descent method
def line_search(system, X, direction, alpha_init=1.0, c=0.5, rho=0.5):
    """
    Backtracking line search to find an appropriate step size.
    system: the nonlinear system function
    X: current point
    direction: search direction
    alpha_init: initial step size
    c, rho: parameters for backtracking
    """
    alpha = alpha_init
    while norm(system(X + alpha * direction)) > (1 - c * alpha) * norm(system(X)):
        alpha *= rho
    return alpha

# Steepest Descent with Line Search
def steepest_descent_line_search(system, X0, tol=1e-6, max_iter=100):
    X = np.array(X0, dtype=float)
    for i in range(max_iter):
        F = system(X)
        if norm(F) < tol:
            return X, i+1
        
        # Use the system's value as an approximation of the gradient
        direction = -F
        alpha = line_search(system, X, direction)
        X = X + alpha * direction
    
    return None, max_iter

# Hybrid method with line search in the steepest descent phase
def hybrid_method_line_search(system, jacobian, X0, tol_sd=5e-2, tol_newton=1e-6, max_iter=100):
    # First use steepest descent with line search
    X, sd_iters = steepest_descent_line_search(system, X0, tol=tol_sd, max_iter=max_iter)
    if X is None:
        return None, sd_iters
    
    # Use the result from steepest descent as the initial guess for Newton's method
    X, newton_iters = newton_method(system, jacobian, X, tol=tol_newton, max_iter=max_iter)
    
    return X, sd_iters + newton_iters

# Initial guess
initial_guess = [0.5, 0.5, 0.5]

# Run the methods again with the new line search implementation
results = {}

# Steepest descent with line search
sd_ls_solution, sd_ls_iters = steepest_descent_line_search(system, initial_guess)

# Hybrid method with line search
hybrid_ls_solution, hybrid_ls_iters = hybrid_method_line_search(system, jacobian, initial_guess)

# Store the results
results['Steepest Descent (Line Search)'] = (sd_ls_solution, sd_ls_iters)
results['Hybrid (Line Search)'] = (hybrid_ls_solution, hybrid_ls_iters)

# Display the results
df_results = pd.DataFrame(results, index=['Solution', 'Iterations'])
df_results

,Steepest Descent (Line Search),Hybrid (Line Search)
Solution,"[0.0, 0.10000006909804049, 1.0000000660275838]","[-5.228141460339611e-17, 0.1000000000097411, 1..."
Iterations,6,6
